In [1]:
# import sys
# !{sys.executable} -m pip install pandas;

In [2]:
import pandas as pd
import time

In [50]:
df = pd.DataFrame(pd.read_csv("../logs/arbbot.csv"))
df["humanTimestamp"] = pd.to_datetime(df['blockTimestamp'], unit='s').dt.strftime('%Y-%m-%d %H:%M:%S')
df["latency"] = df["finishTimestamp"] - df["blockTimestamp"]

In [51]:
# Limit to hour span
hour_limit = 24
sec_limit = hour_limit * 3600
df = df[df.blockTimestamp > time.time()-sec_limit]

## Best opportunities so far

In [52]:
df.sort_values(by="profit", ascending=0).head(10)

,blockNumber,blockTimestamp,finishTimestamp,name,profit,optimalAmount,humanTimestamp,latency
27983,11528966,1608980907,1608980924,kp3rweth_ssuni_r,0.13010,12.270,2020-12-26 11:08:27,17
27901,11528903,1608980095,1608980098,sushiweth_ssuni_r,0.06660,6.052,2020-12-26 10:54:55,3
23373,11525950,1608941851,1608941872,kp3rweth_ssuni_o,0.06387,9.005,2020-12-26 00:17:31,21
26113,11527644,1608963891,1608963903,sushiweth_ssuni_r,0.03455,4.380,2020-12-26 06:24:51,12
25168,11527083,1608956794,1608956807,kp3rweth_ssuni_r,0.03113,6.054,2020-12-26 04:26:34,13
22672,11525684,1608937959,1608937992,linkweth_ssuni_o,0.02281,18.250,2020-12-25 23:12:39,33
24128,11526255,1608945726,1608945739,wethinj_ssuni_r,0.01884,1.726,2020-12-26 01:22:06,13
22041,11524945,1608927780,1608927817,kp3rweth_ssuni_r,0.01826,4.826,2020-12-25 20:23:00,37
22037,11524943,1608927739,1608927777,kp3rweth_ssuni_r,0.01826,4.826,2020-12-25 20:22:19,38
22039,11524944,1608927776,1608927781,kp3rweth_ssuni_r,0.01826,4.826,2020-12-25 20:22:56,5


## Statistics

In [53]:
# Limit opportuninities to the ones beyond the threshold to account for gas costs
threshold_profit = 0.015
above_threshold_df = df[df.profit > threshold_profit]
opp_count = above_threshold_df.blockNumber.count()
mean_latency = above_threshold_df.latency.mean()
max_profit = above_threshold_df.profit.max()
mean_profit = above_threshold_df.profit.mean()
total_profit = above_threshold_df.profit.sum()

In [54]:
print(f"Opportunitys count: {opp_count}")
print(f"Latency average: {mean_latency:.0f} sec")
print(f"Max profit: {max_profit:.2f} ETH")
print(f"Average profit: {mean_profit:.2f} ETH")
print(f"Summed profit: {total_profit:.2f} ETH")

Opportunitys count: 23
Latency average: 14 sec
Max profit: 0.13 ETH
Average profit: 0.03 ETH
Summed profit: 0.64 ETH


## Instruction analysis

In [55]:
opp_comp_df = df.groupby("name").agg({"blockNumber": "count", "profit": ["max", "sum", "mean"]})
opp_comp_df.columns = ["count", "max_profit", "summed_profit", "mean_profit"]
opp_comp_df.index.name = None
opp_comp_df.sort_values(by="mean_profit", ascending=0)

,count,max_profit,summed_profit,mean_profit
snxweth_ssuni_r,27,0.015880,0.080941,0.002998
yfiweth_ssuni_r,18,0.011890,0.047428,0.002635
wethinj_ssuni_o,2604,0.016470,6.368879,0.002446
bandweth_ssuni_r,1395,0.009517,3.282048,0.002353
snxweth_ssuni_o,60,0.009322,0.139153,0.002319
kp3rweth_ssuni_r,691,0.130100,1.429750,0.002069
bandweth_ssuni_o,1454,0.006613,2.848945,0.001959
yfiweth_ssuni_o,22,0.016250,0.042741,0.001943
kp3rweth_ssuni_o,340,0.063870,0.649365,0.001910
linkweth_ssuni_o,63,0.022810,0.113425,0.001800
